## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.86800539e-01, 1.00974910e+00, 9.80245840e-01, 3.93295147e+00,
       4.71736652e+00, 2.24255647e+00, 5.82933404e+00, 3.22644457e+00,
       1.62347586e+00, 9.97204934e+00, 7.56995452e+00, 2.83564462e+00,
       3.43321807e+00, 9.75952265e+00, 5.07553928e+00, 6.90259362e+00,
       3.94598574e+00, 1.75137452e+01, 4.59649254e+00, 1.97809806e+01,
       1.29438590e+01, 4.44815625e+00, 6.42382289e+00, 1.99011582e+01,
       8.07779807e+00, 2.09685216e+01, 1.55628330e+01, 2.61070306e+01,
       1.31810710e+01, 9.30614892e+00, 2.93525976e+01, 2.39994078e+00,
       1.48041123e+01, 5.92887786e+00, 7.07196424e-01, 1.40979526e-02,
       2.15943437e+01, 1.88259007e+01, 2.11260692e+01, 3.48904689e+01,
       2.08767122e+01, 2.29238635e+01, 2.56330384e+01, 5.97658356e+00,
       1.02192022e+01, 2.56473721e+01, 2.43171683e+01, 2.11231070e+01,
       4.31550202e+01, 3.33957172e+01, 3.54679355e+01, 3.64219278e+01,
       1.75829033e+01, 5.09580708e+01, 2.24467020e+01, 2.08453647e-01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(5)

[0.3868005391946976,
 1.0097491040318995,
 0.9802458401469416,
 3.9329514669159913,
 4.71736651708317]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.3558564960591218,
 0.9289691757093477,
 0.9018261729351863,
 3.618315349562712,
 4.339977195716517,
 2.0631519568646435,
 5.362987320774705,
 2.9683290023242512,
 1.4935977919724381,
 9.174285394343222]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [16]:
discounted.take(10)

[0.32027084645320963,
 0.8360722581384129,
 0.8116435556416677,
 3.2564838146064408,
 3.905979476144865,
 1.856836761178179,
 4.826688588697235,
 2.671496102091826,
 1.3442380127751943,
 8.2568568549089]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[779.814584505577,
 776.2094087756288,
 767.4860276633137,
 753.4505352753422,
 727.6346847253056,
 715.2155126498462,
 713.0699704968985,
 711.8003593499328,
 700.4087298431832,
 700.26985443617,
 687.5650159996206,
 680.7586640912067,
 678.1483553865913,
 678.0820049100964,
 672.4933789731882]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [18]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [19]:
mapped = price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.3868005391946976, 0.32027084645320963), (1.0097491040318995, 0.8360722581384129), (0.9802458401469416, 0.8116435556416677), (3.9329514669159913, 3.2564838146064408), (4.71736651708317, 3.905979476144865), (2.242556474852873, 1.856836761178179), (5.829334044320331, 4.826688588697235), (3.2264445677437514, 2.671496102091826), (1.6234758608396067, 1.3442380127751943), (9.972049341677415, 8.2568568549089)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [20]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.3868005391946976, 0.32027084645320963, 1.0097491040318995, 0.8360722581384129, 0.9802458401469416, 0.8116435556416677, 3.9329514669159913, 3.2564838146064408, 4.71736651708317, 3.905979476144865]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [21]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

272

In [22]:
selected_items.take(5)

[300.9555033074961,
 307.35785285325716,
 313.44988892428495,
 315.4891243479075,
 326.1179172027365]

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [23]:
selected_items.reduce(lambda x,y: x + y)

123272.91743055897

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [24]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(42, 300.9555033074961),
 (8, 307.35785285325716),
 (27, 313.44988892428495),
 (35, 315.4891243479075),
 (49, 326.1179172027365),
 (10, 341.7849783514153),
 (23, 344.6968584629858)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [25]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x + y)
total_spent.take(10)

[(40, 1600.839702168622),
 (10, 4854.2727820445125),
 (20, 369.88664116482994),
 (50, 1495.9291860426622),
 (30, 1576.044782151572),
 (21, 1746.0187855644685),
 (31, 2607.8025217554946),
 (1, 2627.978778969345),
 (11, 2464.2530497608204),
 (41, 2818.3152221421415)]

In [26]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(36, 5227.789539407371),
 (37, 5218.448619628508),
 (10, 4854.2727820445125),
 (3, 4455.125550871169),
 (34, 4273.471793758453),
 (13, 4028.9624469876744),
 (45, 3573.319968039256),
 (5, 3413.6093684042194),
 (8, 3246.4695660251336),
 (29, 3231.188541230194),
 (4, 3145.431732427103),
 (47, 3073.9200697516144),
 (26, 3064.3128894599404),
 (24, 2921.1629399245876),
 (46, 2887.110084094291),
 (41, 2818.3152221421415),
 (42, 2752.352514008951),
 (7, 2741.9965159062112),
 (15, 2714.1250403339286),
 (35, 2694.258512724668),
 (17, 2691.999854944879),
 (23, 2650.9558273130906),
 (2, 2646.8442138416663),
 (44, 2637.792923708159),
 (1, 2627.978778969345),
 (31, 2607.8025217554946),
 (48, 2581.310136149059),
 (11, 2464.2530497608204),
 (22, 2452.771723511136),
 (28, 2319.280401477949),
 (25, 2193.56765721847),
 (16, 2084.5280116769777),
 (6, 2061.4353246274204),
 (18, 2053.3144776279723),
 (19, 1873.9149573349275),
 (21, 1746.0187855644685),
 (33, 1699.1979236904608),
 (39, 1663.466689759433),
 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [30]:
sales_data.take(5)

[(47, 300.9555033074961),
 (39, 307.35785285325716),
 (16, 313.44988892428495),
 (43, 315.4891243479075),
 (22, 326.1179172027365)]

In [34]:
total_items = sales_data.countByKey()
total_items.items()

dict_items([(33, 3), (31, 6), (24, 10), (44, 6), (38, 4), (23, 10), (27, 4), (6, 7), (50, 4), (26, 5), (13, 6), (39, 7), (12, 5), (2, 7), (5, 5), (47, 3), (4, 9), (43, 10), (36, 4), (34, 9), (45, 8), (10, 7), (42, 4), (16, 6), (15, 6), (18, 7), (14, 4), (41, 4), (19, 3), (7, 6), (17, 6), (30, 6), (11, 6), (28, 7), (32, 4), (29, 4), (46, 5), (20, 6), (1, 5), (49, 4), (22, 7), (37, 6), (3, 5), (21, 2), (8, 6), (35, 5), (25, 4), (48, 5)])

In [36]:
sorted(total_items.items(), key=lambda kv:kv[1],reverse=True)

[(24, 10),
 (23, 10),
 (43, 10),
 (4, 9),
 (34, 9),
 (45, 8),
 (6, 7),
 (39, 7),
 (2, 7),
 (10, 7),
 (18, 7),
 (28, 7),
 (22, 7),
 (31, 6),
 (44, 6),
 (13, 6),
 (16, 6),
 (15, 6),
 (7, 6),
 (17, 6),
 (30, 6),
 (11, 6),
 (20, 6),
 (37, 6),
 (8, 6),
 (26, 5),
 (12, 5),
 (5, 5),
 (46, 5),
 (1, 5),
 (3, 5),
 (35, 5),
 (48, 5),
 (38, 4),
 (27, 4),
 (50, 4),
 (36, 4),
 (42, 4),
 (14, 4),
 (41, 4),
 (32, 4),
 (29, 4),
 (49, 4),
 (25, 4),
 (33, 3),
 (47, 3),
 (19, 3),
 (21, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
